In [30]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [31]:
# https://www.kaggle.com/new-york-city/nyc-property-sales

In [50]:
# Read csv file
property_data = pd.read_csv("Resources/nyc-rolling-sales.csv")
property_data.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


In [51]:
# Only show needed columns
property_data = property_data[["BOROUGH", "NEIGHBORHOOD", "ADDRESS","SALE PRICE"]]
property_data

,BOROUGH,NEIGHBORHOOD,ADDRESS,SALE PRICE
0,1,ALPHABET CITY,153 AVENUE B,6625000
1,1,ALPHABET CITY,234 EAST 4TH STREET,-
2,1,ALPHABET CITY,197 EAST 3RD STREET,-
3,1,ALPHABET CITY,154 EAST 7TH STREET,3936272
4,1,ALPHABET CITY,301 EAST 10TH STREET,8000000
...,...,...,...,...
84543,5,WOODROW,37 QUAIL LANE,450000
84544,5,WOODROW,32 PHEASANT LANE,550000
84545,5,WOODROW,49 PITNEY AVENUE,460000
84546,5,WOODROW,2730 ARTHUR KILL ROAD,11693337


In [52]:

# Replace - with 0
property_sales_df = property_data.replace("-",0, regex=True)

In [53]:
# Check data types
property_sales_df.dtypes

BOROUGH          int64
NEIGHBORHOOD    object
ADDRESS         object
SALE PRICE      object
dtype: object

In [54]:
# Drop any rows with sale price $0
property_sales_df = property_sales_df.replace(0, np.nan)
property_sales_df = property_sales_df.dropna(how='any', axis=0)
property_sales_df.head()

,BOROUGH,NEIGHBORHOOD,ADDRESS,SALE PRICE
0,1,ALPHABET CITY,153 AVENUE B,6625000
3,1,ALPHABET CITY,154 EAST 7TH STREET,3936272
4,1,ALPHABET CITY,301 EAST 10TH STREET,8000000
6,1,ALPHABET CITY,210 AVENUE B,3192840
9,1,ALPHABET CITY,629 EAST 5TH STREET,16232000


In [55]:
# Change data type to integer
property_sales_df["SALE PRICE"] = pd.to_numeric(property_sales_df["SALE PRICE"])
property_sales_df.head()

,BOROUGH,NEIGHBORHOOD,ADDRESS,SALE PRICE
0,1,ALPHABET CITY,153 AVENUE B,6625000
3,1,ALPHABET CITY,154 EAST 7TH STREET,3936272
4,1,ALPHABET CITY,301 EAST 10TH STREET,8000000
6,1,ALPHABET CITY,210 AVENUE B,3192840
9,1,ALPHABET CITY,629 EAST 5TH STREET,16232000


In [56]:
# Confirm data type
property_sales_df.dtypes

BOROUGH          int64
NEIGHBORHOOD    object
ADDRESS         object
SALE PRICE       int64
dtype: object

In [39]:
property_sales_df = property_sales_df.rename(columns = {"BOROUGH" : "borough_id",
                                                        "NEIGHBORHOOD" : "neighborhood",
                                                        "ADDRESS" : "address",
                                                        "SALE PRICE" : "sale_price"})
property_sales_df

,borough_id,neighborhood,address,sale_price
0,1,ALPHABET CITY,153 AVENUE B,6625000
3,1,ALPHABET CITY,154 EAST 7TH STREET,3936272
4,1,ALPHABET CITY,301 EAST 10TH STREET,8000000
6,1,ALPHABET CITY,210 AVENUE B,3192840
9,1,ALPHABET CITY,629 EAST 5TH STREET,16232000
...,...,...,...,...
84543,5,WOODROW,37 QUAIL LANE,450000
84544,5,WOODROW,32 PHEASANT LANE,550000
84545,5,WOODROW,49 PITNEY AVENUE,460000
84546,5,WOODROW,2730 ARTHUR KILL ROAD,11693337


In [40]:
borough_table = pd.DataFrame({  'borough_id' : [1,2,3,4,5],
                                'borough':['Manhattan','Bronx','Brooklyn','Queens','Staten Island']})
borough_table

,borough_id,borough
0,1,Manhattan
1,2,Bronx
2,3,Brooklyn
3,4,Queens
4,5,Staten Island


In [41]:
property_sales_df = pd.merge(borough_table, property_sales_df, on = 'borough_id')
property_sales_df

,borough_id,borough,neighborhood,address,sale_price
0,1,Manhattan,ALPHABET CITY,153 AVENUE B,6625000
1,1,Manhattan,ALPHABET CITY,154 EAST 7TH STREET,3936272
2,1,Manhattan,ALPHABET CITY,301 EAST 10TH STREET,8000000
3,1,Manhattan,ALPHABET CITY,210 AVENUE B,3192840
4,1,Manhattan,ALPHABET CITY,629 EAST 5TH STREET,16232000
...,...,...,...,...,...
39884,5,Staten Island,WOODROW,37 QUAIL LANE,450000
39885,5,Staten Island,WOODROW,32 PHEASANT LANE,550000
39886,5,Staten Island,WOODROW,49 PITNEY AVENUE,460000
39887,5,Staten Island,WOODROW,2730 ARTHUR KILL ROAD,11693337


In [45]:
# Create engine
rds_connection_string = "postgres:postgres@localhost:5432/etl_project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [46]:
# Get table names
engine.table_names()

['property_sales']

In [47]:
# Load property_sales_df data to sql table
property_sales_df.to_sql(name='property_sales', con=engine, if_exists='append', index=False)


In [48]:
pd.read_sql_query('select * from property_sales', con=engine)

,id,borough_id,borough,neighborhood,address,sale_price
0,1,1,Manhattan,ALPHABET CITY,153 AVENUE B,6625000
1,2,1,Manhattan,ALPHABET CITY,154 EAST 7TH STREET,3936272
2,3,1,Manhattan,ALPHABET CITY,301 EAST 10TH STREET,8000000
3,4,1,Manhattan,ALPHABET CITY,210 AVENUE B,3192840
4,5,1,Manhattan,ALPHABET CITY,629 EAST 5TH STREET,16232000
...,...,...,...,...,...,...
39884,39885,5,Staten Island,WOODROW,37 QUAIL LANE,450000
39885,39886,5,Staten Island,WOODROW,32 PHEASANT LANE,550000
39886,39887,5,Staten Island,WOODROW,49 PITNEY AVENUE,460000
39887,39888,5,Staten Island,WOODROW,2730 ARTHUR KILL ROAD,11693337
